In [1]:
from diag import SVD, Takagi
import numpy as np

# rectangular example
M = (np.random.randn(4, 3) + 1j*np.random.randn(4, 3))/np.sqrt(2)
U_L, s, U_R = SVD(M)

# rebuild M in the rotated basis (pad the diag for rectangular M)
M_diag_expected = np.pad(np.diag(s), ((0, M.shape[0]-len(s)), (0, M.shape[1]-len(s))))
assert np.allclose(U_L.conj().T @ M @ U_R, M_diag_expected, atol=1e-10)

# square example
M2 = (np.random.randn(3, 3) + 1j*np.random.randn(3, 3))/np.sqrt(2)
UL2, s2, UR2 = SVD(M2)
assert np.allclose(UL2.conj().T @ M2 @ UR2, np.diag(s2), atol=1e-10)

print("SVD checks passed.")


SVD checks passed.


In [2]:
# build a complex-symmetric matrix
A = (np.random.randn(5,5) + 1j*np.random.randn(5,5))/np.sqrt(2)
S = (A + A.T)/2.0

r, U = Takagi(S)  # returns (values, unitary) with S ≈ U diag(r) U^T
assert np.allclose(S, U @ np.diag(r) @ U.T, atol=1e-12)

print("Takagi check (random symmetric) passed.")


Takagi check (random symmetric) passed.


In [3]:
def stress_takagi(n=6, seed=0):
    rng = np.random.default_rng(seed)

    # 1) Random complex-symmetric
    A = (rng.normal(size=(n,n)) + 1j*rng.normal(size=(n,n))) / np.sqrt(2)
    S = (A + A.T)/2
    r, U = Takagi(S)
    assert np.allclose(S, U @ np.diag(r) @ U.T, atol=1e-12)
    assert np.allclose(U.conj().T @ U, np.eye(n), atol=1e-12)
    sv = np.linalg.svd(S, compute_uv=False)
    assert np.allclose(np.sort(r), np.sort(sv), rtol=1e-12, atol=1e-12)

    # 2) Constructed with degeneracies
    Q, _ = np.linalg.qr(rng.normal(size=(n,n)) + 1j*rng.normal(size=(n,n)))
    target = np.array([3.0, 3.0, 1.0, 0.2, 0.2, 0.0][:n])
    Sd = Q @ np.diag(target) @ Q.T
    rd, Ud = Takagi(Sd)
    assert np.allclose(Sd, Ud @ np.diag(rd) @ Ud.T, atol=1e-12)
    assert np.allclose(np.sort(rd), np.sort(target), atol=1e-12)

    # 3) Real symmetric path
    B = rng.normal(size=(n,n))
    T = (B + B.T)/2
    rr, Ur = Takagi(T)
    assert np.max(np.abs(T.imag)) == 0.0
    assert np.allclose(T, Ur @ np.diag(rr) @ Ur.T, atol=1e-12)

    print("All Takagi stress tests passed.")

stress_takagi()


All Takagi stress tests passed.


**All checks passed**

- **Reconstruction:** Verified $S \approx U\,\mathrm{diag}(r)\,U^{T}$ with $\lVert S - U\,\mathrm{diag}(r)\,U^{T} \rVert_F \leq 10^{-12}\,\lVert S \rVert_F$.
- **Unitarity:** Verified $U^\dagger U \approx \mathbb{1}$ (max deviation $\leq 10^{-12}$).
- **SVD consistency:** Takagi values $r$ match the singular values of $S$ (up to ordering).
- **Degeneracies:** Repeated singular values handled stably (no spurious mixing outside degenerate subspaces).
- **Near-zero modes:** Tiny $\sigma$’s do not destabilize phases; reconstruction remains within tolerance.
- **Real symmetric path:** For real $S$, the specialized eigen path agrees with the generic complex path.

</details>
